In [6]:
import cantools
from collections import defaultdict
from IPython.display import Markdown, display

In [7]:
baud_rate = 1000

can1_files = [
    "haltech.dbc",
    "tcs.dbc",
    "swc.dbc",
    "acc.dbc",
    "izze_ttpms.dbc",
    "izze_irts_tire_v2.dbc",
    "izze_imu_v2.dbc",
    "izze_8pa_v1_differential.dbc"
]

can2_files = [
    
]

In [8]:
def message_throughput(message):
    frame_size = 47

    if message.is_extended_frame:
        frame_size += 20

    attr = message.dbc.attributes.get('GenMsgCycleTime')
    if not attr:
        return 0

    cycle_time = int(attr.value)
    if cycle_time == 0:
        return 0

    frame_size += 8 * message.length
    frame_size *= 1.15

    return frame_size * 1000 / cycle_time

def print_throughput(name, bps):
    kbps = bps / 1000
    percentage = round((kbps / baud_rate) * 100, 3)
    print(f"{name:<40} {kbps:>8.2f} kbps {percentage:>8.2f} %")

In [9]:
total_bps_can1 = 0
total_bps_can2 = 0

# CAN 1
display(Markdown(f"**CAN 1 Utilization ({baud_rate} kbps)**"))

for file in can1_files:
    db = cantools.database.load_file(f"../dbc/{file}")
    
    bps = round(sum(message_throughput(msg) for msg in db.messages))
    total_bps_can1 += bps
    print_throughput(file, bps)

print_throughput(f"", total_bps_can1)

# CAN 2
display(Markdown(f"**CAN 2 Utilization ({baud_rate} kbps)**"))

for file in can2_files:
    db = cantools.database.load_file(f"../dbc/{file}")
    
    bps = round(sum(message_throughput(msg) for msg in db.messages))
    total_bps_can2 += bps
    print_throughput(file, bps)

print_throughput("", total_bps_can2)

**CAN 1 Utilization (1000 kbps)**

haltech.dbc                                172.97 kbps    17.30 %
tcs.dbc                                      6.60 kbps     0.66 %
swc.dbc                                     14.14 kbps     1.41 %
acc.dbc                                    150.65 kbps    15.06 %
izze_ttpms.dbc                               2.45 kbps     0.24 %
izze_irts_tire_v2.dbc                       63.83 kbps     6.38 %
izze_imu_v2.dbc                             25.53 kbps     2.55 %
izze_8pa_v1_differential.dbc                86.54 kbps     8.65 %
                                           522.70 kbps    52.27 %


**CAN 2 Utilization (1000 kbps)**

                                             0.00 kbps     0.00 %


In [10]:
frame_map = defaultdict(list)

for file in can1_files:
    db = cantools.database.load_file(f"../dbc/{file}")

    for msg in db.messages:
        frame_id = msg.frame_id

        frame_map[frame_id].append((file, msg.name))

found = False
for frame_id, entries in frame_map.items():
    if len(entries) > 1:
        found = True
        print(f"Duplicate ID 0x{frame_id:X}:")
        for file, name in entries:
            print(f" - {file}: {name}")
        print()

if not found:
    print("No duplicate message IDs")

No duplicate message IDs
